In [8]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from sacrebleu.metrics import BLEU
import ast

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased", cache_dir="/mntssd/mnt3/shanshanbai/my_storage_from_qian/.cache/huggingface/hub")



In [9]:
# Load dataset
df = pd.read_csv('/mntssd/mnt3/shanshanbai/nlpinearthobservation/synthetic_data/data/synthetic.csv')
# df = pd.read_csv('/mntssd/mnt3/shanshanbai/my_storage_from_qian/results/generated tweets/real.csv')

# # Undersample the dataset for balanced class distribution
# min_class_size = 3000
# df = (
#     df.groupby('mapped_class')
#     .apply(lambda x: x.sample(n=min_class_size, random_state=42))
#     .reset_index(drop=True)
# )

# print("Class distribution after undersampling:\n", df['mapped_class'].value_counts())

# Process tweets
df['tweets'] = df['tweets'].apply(ast.literal_eval) # tweet_no_url
all_tweets = [tweet for sublist in df['tweets'] for tweet in sublist]


In [10]:
len(all_tweets)

16098

In [12]:
tokenized_tweets = [" ".join(tokenizer.tokenize(tweet)) for tweet in all_tweets]

In [14]:
# Initialize BLEU scorer
bleu = BLEU(effective_order=True)

In [15]:
# Function to compute self-BLEU
def compute_self_bleu(tokenized_texts, n_gram=4):
    scores = []
    
    for i, ref in enumerate(tokenized_texts):
        references = [t for j, t in enumerate(tokenized_texts) if i != j]  # Exclude current tweet
        score = bleu.corpus_score([ref], [references]).score  # Compute BLEU score
        scores.append(score)

    return np.mean(scores)  # Average self-BLEU

In [16]:
# Compute 4-gram self-BLEU
self_bleu_score = compute_self_bleu(tokenized_tweets, n_gram=4)

print(f"4-gram Self-BLEU Score: {self_bleu_score:.2f}")

KeyboardInterrupt: 